In [21]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms

In [22]:
device ='cuda' if torch.cuda.is_available() else "cpu"
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [23]:
#parameters
learning_rate=0.001
training_epochs=15
batch_size=100

In [24]:
#datasets
mnist_train=dsets.MNIST(root='MNIST_data/',train=True,transform=transforms.ToTensor(),download=True) #transform은 tensor value로 전환
mnist_test=dsets.MNIST(root='MNIST_data/',train=False,transform=transforms.ToTensor(),download=True)

In [25]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)

In [26]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1=nn.Sequential(nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
                                 nn.ReLU(),nn.MaxPool2d(2))
        self.layer2=nn.Sequential(nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
                                 nn.ReLU(),nn.MaxPool2d(2))
        self.layer3=nn.Sequential(nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
                                 nn.ReLU(),nn.MaxPool2d(2))
        self.fc1=nn.Linear(3*3*128,625,bias=True)
        self.ReLU=nn.ReLU()
        self.fc2=nn.Linear(625,10,bias=True) #7 by 7의 64채널을 0~9까지의 10개의 레이블로
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
       
        
        out=out.view(out.size(0),-1) #out을 view로 펼침, batch size로 펼치고 나머지는 한줄로
        out=self.fc1(out)
        out=self.ReLU(out)
        out=self.fc2(out)
        return out

In [27]:
model=CNN().to(device)

In [28]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [29]:
#training
total_batch=len(data_loader) #전체 batch의 개수 
print("Learning start")

for epoch in range(training_epochs):
    avg_cost=0 #loss를 하나씩 담을 것
    
    for X,Y in data_loader: #X는 이미지, Y는 label
        X=X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        
        cost=criterion(hypothesis,Y) #가설과 실제정답 차이를 계산
        cost.backward()
        
        optimizer.step()
        
        avg_cost+=cost/total_batch
    print("Epoch:{} | cost={}".format(epoch+1,avg_cost))
print("Learning finished")

Learning start
Epoch:1 | cost=0.1601768136024475
Epoch:2 | cost=0.04240937530994415
Epoch:3 | cost=0.03040977567434311


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1) ==Y_test
    accuracy=correct_prediction.float().mean()
    print("Accuracy:",accuracy.item())